In [1]:
# Common modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.feature_selection import mutual_info_classif, SelectKBest

# Decision tree specific modules
from sklearn import tree

from scipy.sparse import load_npz

from sklearn.feature_extraction.text import CountVectorizer #Can use tfidffvectorizer as well
import pandas as pd 
import os
import re
import numpy as np

from matplotlib import pyplot as plt
from collections import defaultdict, Counter

In [2]:
import xgboost as xgb

In [3]:
BASE_PATH = os.path.join(os.getcwd(), os.pardir)
DATA_PATH = os.path.join(BASE_PATH, 'data')

In [4]:
X_train = load_npz(os.path.join(DATA_PATH, 'training_feats.npz'))

In [5]:
X_test = load_npz(os.path.join(DATA_PATH, 'test_feats.npz'))

In [6]:
X_train.shape

(49308, 35522)

In [7]:
X_test.shape

(74659, 35522)

In [8]:
train_df = pd.read_json(os.path.join(BASE_PATH, '01-milestone1', 'imputed_train.json'))
test_df =  pd.read_json(os.path.join(DATA_PATH, 'test.json.zip'))

In [9]:
y_train = train_df['interest_level']

In [10]:
X_test_indexes = test_df.listing_id

In [11]:
# Convert labels from {low, medium high} -> {0, 1, 2}
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

## Train-test split

We need to split the training data into a training set and a validation set (to evaluate the performance of the model).

For now we do 5-fold cross_validation.

In [12]:
seed = 36201431

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [14]:
dtrain

In [24]:
# Parameters
param = {'max_depth': 2, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 3}
param['nthread'] = 4
param['eval_metric'] = 'mlogloss'

In [25]:
num_round = 10
bst = xgb.cv(param, dtrain, num_round, nfold=5)

In [26]:
bst

,train-mlogloss-mean,train-mlogloss-std,test-mlogloss-mean,test-mlogloss-std
0,0.762534,0.001187,0.762881,0.004279
1,0.711617,0.001480,0.712871,0.003569
2,0.683942,0.002275,0.685679,0.005683
3,0.668774,0.000842,0.671859,0.004634
4,0.659361,0.000404,0.662255,0.003968
5,0.649568,0.001146,0.653808,0.004605
6,0.643176,0.000818,0.647342,0.004386
7,0.637702,0.000772,0.642356,0.004204
8,0.632530,0.001023,0.638238,0.003732
9,0.628463,0.000823,0.634779,0.003829


In [14]:
# def create_submission_csv(y_pred, X_test_indexes):
#     df = pd.DataFrame(y_pred, columns=le.classes_)
#     df.index = X_test_indexes
#     df.index.name = 'listing_id'
#     return df

In [27]:
# output = create_submission_csv(y_pred, X_test_indexes)
# output.to_csv('decision_tree_predictions_1.csv')